# HydroTurbine + Reservoir for WaterModel

> **Set up**
>
> To run this notebook, first install the Julia kernel for Jupyter Notebooks using [IJulia](https://julialang.github.io/IJulia.jl/stable/manual/installation/), then [create an environment](https://pkgdocs.julialang.org/v1/environments/) for this tutorial with the packages listed with `using <PackageName>` further down.
>
> This tutorial has demonstrated compatibility with these package versions. If you run into any errors, first check your package versions for consistency using `Pkg.status()`.
>
 > ```
 > Status `~/work/HydroPowerSimulations.jl/HydroPowerSimulations.jl/docs/Project.toml`
 >   [a93c6f00] DataFrames v1.8.1
 >   [864edb3b] DataStructures v0.19.3
 >   [e30172f5] Documenter v1.16.1
 >   [d12716ef] DocumenterInterLinks v1.1.0
 >   [35a29f4d] DocumenterTools v0.1.21
 >   [87dc4568] HiGHS v1.21.0
 >   [fc1677e0] HydroPowerSimulations v0.13.1 `~/work/HydroPowerSimulations.jl/HydroPowerSimulations.jl`
 >   [2cd47ed4] InfrastructureSystems v3.3.0
 >   [b6b21f68] Ipopt v1.14.0
 >   [23fbe1c1] Latexify v0.16.10
 >   [98b081ad] Literate v2.21.0
 > ⌅ [94fada2c] PowerFlows v0.13.1
 >   [e690365d] PowerSimulations v0.32.4
 >   [f00506e0] PowerSystemCaseBuilder v2.2.0
 >   [bcd98974] PowerSystems v5.4.0
 >   [08abe8d2] PrettyTables v3.1.2
 >   [9e3dc215] TimeSeries v0.25.2
 > Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`
 > 
 > ```


> *Note*
>
>
> `HydroPowerSimulations.jl` is an extension library of [`PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/) for modeling hydro units. Users are encouraged to review the [single-step tutorial in `PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/tutorials/decision_problem/) before this tutorial.
>
## Load packages

In [ ]:
using PowerSystems
using PowerSimulations
using HydroPowerSimulations
using PowerSystemCaseBuilder
using Ipopt ## solver

## Data

> *Note*
>
>
> `PowerSystemCaseBuilder.jl` is a helper library that makes it easier to reproduce examples in the documentation and tutorials. Normally you would pass your local files to create the system data instead of calling the function `build_system`.
> For more details visit [PowerSystemCaseBuilder README](https://github.com/NREL-Sienna/PowerSystemCaseBuilder.jl/blob/main/README.md)

In [ ]:
sys = build_system(PSITestSystems, "c_sys5_hy_turbine_head")

With a single `PowerSystems.HydroTurbine` connected downstream to a `PowerSystems.HydroReservoir`:

In [ ]:
hy = only(get_components(HydroTurbine, sys))

res = only(get_components(HydroReservoir, sys))

Note that the reservoir has a `level_data_type` of `HEAD`, that implies its storage level limits data are in meters (above the sea level) and refer to the hydraulic head levels. That means that its available capacity lies with its hydraulic head being within 463.5 and 555.5 meters, and its intake elevation is at 463.3 meters. In addition note that the elevation of the turbine is on 317.12 meters above the sea level.

## Decision Model

Setting up the formulations based on [`PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/formulation_library/Introduction/):

In [ ]:
template = ProblemTemplate(PTDFPowerModel)
set_device_model!(template, ThermalStandard, ThermalBasicDispatch)
set_device_model!(template, PowerLoad, StaticPowerLoad)
set_device_model!(template, Line, StaticBranch)

but, now we also include the HydroTurbine using `HydroTurbineBilinearDispatch`:

In [ ]:
set_device_model!(template, HydroTurbine, HydroTurbineBilinearDispatch)

This is a nonlinear model that to compute its output power requires the bilinear term `head` times `water flow`. For that purpose the non-convex Ipopt solver will be used to solve this problem.

In addition, we need to use the water model for the HydroReservoir via `HydroWaterModelReservoir`.

In [ ]:
set_device_model!(template, HydroReservoir, HydroWaterModelReservoir)

With the template properly set-up, we construct, build and solve the optimization problem:

In [ ]:
model = DecisionModel(template, sys; optimizer = Ipopt.Optimizer)
build!(model; output_dir = mktempdir())
solve!(model)

## Exploring Results

Results can be explored using:

In [ ]:
res = OptimizationProblemResults(model)

Use `read_variable` to read in the dispatch variable results for the hydro:

In [ ]:
var =
    read_variable(res, "ActivePowerVariable__HydroTurbine"; table_format = TableFormat.WIDE)

or the water flowing through the turbine (in m³/s):

In [ ]:
var = read_expression(
    res,
    "TotalHydroFlowRateTurbineOutgoing__HydroTurbine";
    table_format = TableFormat.WIDE,
)

and the head level of the reservoir:

In [ ]:
hydraulic_head = read_variable(
    res,
    "HydroReservoirHeadVariable__HydroReservoir";
    table_format = TableFormat.WIDE,
)

Note that since the water outflow limit of the turbine is limited on 30 m³/s, the optimal solution decides to flow as much water as possible producing power around 190 MW with that flow and hydraulic head.